<a href="https://colab.research.google.com/github/tonystz/gitpod/blob/main/pycuda_workshop.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction to CUDA and PyCUDA

In [ ]:
import numpy as np
import pandas as pd


df=pd.read_csv('drive/MyDrive/a.csv').dropna()
print(df)
a= df.to_numpy()
print(a)
print(a.shape,a.size,a.dtype)

# python to check risk IP, check threshhold: >5 times within 6 minute

In [ ]:
from datetime import datetime
import csv
import socket, struct
import numpy as np

ip2long=lambda ip:struct.unpack("!L", socket.inet_aton(ip))[0]
long2ip=lambda long:socket.inet_ntoa(struct.pack('!L', long))
date2int=lambda current_date: current_date.year*10000000000 + current_date.month * 100000000 +  current_date.day * 1000000 + current_date.hour*10000 + current_date.minute*100 + current_date.second

#pre-process data
hack_type={
    'Ftp访问日志':0,
    'SQL 注入':1,
    'Web Shell攻击':2,
    '暴力破解':3,
    '端口扫描':4,
    '缓冲区溢出攻击':5,
    '弱口令漏洞':6
}

numdata=[]
with open('drive/MyDrive/a.csv', newline='') as csvfile:
    data = csv.reader(csvfile)
    next(data) #skip header
    for row in data:
        if len(row[0]) == 0: continue
        numdata.append([date2int(datetime.strptime(row[1]+' '+row[2],'%Y/%m/%d %H:%M:%S')),hack_type.get(row[3],-1),ip2long(row[4])])
        print(row)
print('number data',numdata)
a=np.array(numdata,dtype=np.uint64)

# r=np.zeros(a.shape,dtype=a.dtype)
#group data
rdict={}
for i in a:
    if i[2] in rdict:
        rdict[i[2]][0] +=1
        rdict[i[2]][1].append(i[0])
    else:
        rdict[i[2]]=[1,[i[0]]]

print('rdict',rdict)

# check threshhold: >5 within 6 minute
th_count=5
th_min=6*100

for k,v in rdict.items():
    if v[0] > th_count:
        avg_time=int(sum(v[1])/len(v[1]))
        if [i for i in v[1] if i < avg_time + th_min]:
            print(long2ip(k))

# GUP version

In [13]:
!pip install pycuda # install cuda


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [69]:
#%%writefile a.py
import pycuda.driver as cuda
import pycuda.autoinit
from pycuda.compiler import SourceModule
from pycuda import gpuarray
import numpy as np
import pycuda.driver as drv

mod_ker = SourceModule("""
  #include <stdint.h>
  __global__ void group_data(uint32_t *a)
  {
    int idx = threadIdx.x + blockIdx.x * blockDim.x;
    int idy = threadIdx.y + blockIdx.y * blockDim.y;
    printf("loca is %zu:%zu \\n", idx,idy);
    // out[idx][idy]=2;
  }
  """)
group_data_gpu=mod_ker.get_function('group_data')

numdata=np.array(numdata,dtype=np.uint32)
numdata_gpu=gpuarray.to_gpu(numdata)

#loop in CPU

print('size',numdata_gpu.shape)
# out_gpu=gpuarray.empty_like(numdata_gpu)
group_data_gpu(numdata_gpu,block=(1024,1,1), grid=(1,1,1))
# drv.Context.synchronize()
# print(out_gpu)


<ipython-input-69-69c3d8b4ebf6>:9: UserWarning: The CUDA compiler succeeded, but said the following:
kernel.cu(8): warning #181-D: argument is incompatible with corresponding format string conversion

kernel.cu(8): warning #181-D: argument is incompatible with corresponding format string conversion


  mod_ker = SourceModule("""


LogicError: ignored

In [61]:
!python a.py
# build and debug the cuda c code
#!nvcc --cubin -arch sm_75 -I/usr/local/lib/python3.9/dist-packages/pycuda/cuda /tmp/tmp3vuxop3u/kernel.cu


Traceback (most recent call last):
  File "/content/a.py", line 19, in <module>
    numdata=np.array(numdata,dtype=np.uint32)
NameError: name 'numdata' is not defined


In [70]:
#Example: loop 2 dimensional array
import pycuda.driver as cuda
from pycuda.compiler import SourceModule
import numpy as np
import cv2 

import pycuda.autoinit

img = cv2.imread('drive/MyDrive/test.jpg',0)
img_size=img.shape
print(img_size)
print(img.dtype)

#nbtes determines the number of bytes for the numpy array a
img_gpu = cuda.mem_alloc(img.nbytes)
#Copies the memory from CPU to GPU
cuda.memcpy_htod(img_gpu, img)

mod = SourceModule("""
#include <stdio.h>
__global__ void AHE(unsigned char *a, int row, int col)
{
int i = threadIdx.x+ blockIdx.x* blockDim.x;
int j = threadIdx.y+ blockIdx.y* blockDim.y;

if(i==0 && j ==0)
printf("Output array \n");

if(i <row && j < col)
{
    int val = int(a[j + i*col]);
    printf(" %d", val);
}
}
""")
#Gives you the number of columns
col = np.int32(img.shape[-1])
row = np.int32(img.shape[0])
func = mod.get_function("AHE")
func(img_gpu, row, col, block=(32,32,1))
img_ahe = np.empty_like(img)
cuda.memcpy_dtoh(img_ahe, img_gpu)

(355, 662)
uint8


LogicError: ignored

In [ ]:
func = mod.get_function("doublify")
func(a_gpu, block=(4,4,1))

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
a_doubled = numpy.empty_like(a)
cuda.memcpy_dtoh(a_doubled, a_gpu)
print(a_doubled)
print(a)

In [ ]:
b = numpy.random.randn(4,4)
b = b.astype(numpy.float32)
c = numpy.random.randn(4,4)
c = c.astype(numpy.float32)

In [ ]:
mod2 = SourceModule("""
  __global__ void add2(float *a, float *b)
  {
    int idx = threadIdx.x + threadIdx.y*4;
    a[idx] += b[idx];
  }
  """)

In [ ]:
b_gpu = cuda.mem_alloc(b.nbytes)
c_gpu = cuda.mem_alloc(c.nbytes)

cuda.memcpy_htod(b_gpu, b)
cuda.memcpy_htod(c_gpu, c)


In [ ]:
func = mod2.get_function("add2")
func(b_gpu,c_gpu, block=(4,4,1))

In [ ]:
added = numpy.empty_like(b)
cuda.memcpy_dtoh(added, b_gpu)
print(added)
print(b)
print(c)

# Exercises

1. Write a cuda kernel to find the elementwise square of a matrix
2. Write a cuda kernel to find a matrix, which when added to the given matrix results in every element being equal to zero
3. Write a cuda kernel to multiply two matrices:
    1. Assume square matrices, with dimensions < 1024
    2. Assume square matrices, with dimensions > 1024
    3. Assume non-square matrices, with dimensions > 1024

In [ ]:
1. 